In [ ]:
import psi4
from psi4 import *
from psi4.core import *
import numpy as np
import os
sys.path.append('os.getcwd()')
from opt_helper import stre, bend, intcosMisc, linearAlgebra

## Rational Function Optimization

Rational Function Optimization (RFO) is the method of choice for minimizations. This tutorial will walk through the basic theory of RFO and show a sample calculation.  The method was introduced for geometry optimizations by A. Banerjee, N. Adams, J. Simons, and R. Shepard in _J. Phys. Chem._ 89, 52 (1985).

In the Newton-Raphson method, the potential energy surface is approximated by the truncated Taylor expansion in internal coordinates $q$ and gradient $g$ (where vectors are interpreted as columns).

$$ \epsilon = E(q) - E_0 = g^T \Delta q + \frac{1}{2}(\Delta q)^T \mathbf{H} \Delta q $$

An extension is to express the potential via a [2/2] Pade approximation, where __S__ is the scaling matrix.  If __S__ were zero, then the harmonic approximation would be obtained.

$$ \epsilon = \frac{g^T\Delta q + \frac{1}{2} (\Delta q)^T \textbf{H} \Delta q}{1 + \Delta q^T \textbf{S} \Delta q} $$


The relative energy expression can be rewritten in the form of $N+1$ dimensional vectors, where $N$ is the number of coordinates.

$$ \epsilon = \frac{ \frac{1}{2}\begin{pmatrix} \Delta{q^T} & 1\end{pmatrix}\begin{pmatrix} {\textbf H} & g \\ g^T & 0  \end{pmatrix}\begin{pmatrix}\Delta{q} \\ 1\end{pmatrix} }{ \begin{pmatrix} \Delta q^T & 1 \end{pmatrix} \begin{pmatrix} \textbf S & 0  \\ 0 & 1 \end{pmatrix} \begin{pmatrix} \Delta q \\ 1 \end{pmatrix}}$$

since the right-hand side is
\begin{align}
  &= \frac{ \frac{1}{2}\begin{pmatrix} \Delta{q^T} & 1\end{pmatrix} \begin{pmatrix} \textbf{H} \Delta q  + g \\ g^T \Delta q + 0 \end{pmatrix}}{\begin{pmatrix} \Delta q^T & 1 \end{pmatrix} \begin{pmatrix} \textbf S \Delta q + 0 \\ 0 + 1 \end{pmatrix}} \\
  \\
 &= \frac { \frac{1}{2} \Delta q^T \textbf H \Delta q + \frac{1}{2} \Delta q^T g + \frac{1}{2} g^T \Delta q} { \Delta q^T\textbf S \Delta q + 1}
 \\
\end{align}
which is equivalent to the expression above.

Making the stationary point assumption that  
$$\frac{\partial \epsilon}{\partial q } = 0 $$
we can derive the expression for the step.

\begin{align}
\frac{\partial \epsilon}{\partial q} &= \frac {g + \mathbf{H} \Delta q}{1 + \Delta q^T \textbf S \Delta q} - \frac{ g \Delta q^T + \frac{1}{2} \Delta q^T \textbf H \Delta q}{ 1 + \Delta q^T \textbf S \Delta q} \Big( \frac{ 2 \textbf S \Delta q}{1 + \Delta q^T \textbf S \Delta q}\Big) \\
\\
\frac{\partial \epsilon}{\partial q} &= \frac {g + \textbf{H}\Delta q}{1 + \Delta q^T \textbf S \Delta q} - \epsilon \Big( \frac{ 2 \textbf S \Delta q}{1 + \Delta q^T \textbf S \Delta q} \Big) \\
\\
0 &= \frac{ g + \textbf{H} \Delta q - 2 \epsilon \textbf S \Delta q}{1 + \Delta q^T \textbf S \Delta q}\\
\\
0 &=  g + \textbf{H} \Delta q - 2 \epsilon \textbf S \Delta q \\
\\
g + H\Delta q &= 2 \epsilon \textbf S \Delta q = \lambda \textbf S \Delta q\\
\end{align}

where $\lambda$ is defined as $2\epsilon$.  It can be shown that 
$$\lambda = g^T \Delta q$$

which allows the stationarity condition to be written as follows.

\begin{align}
\begin{pmatrix} \textbf H \Delta q \\ g^T \Delta q \end{pmatrix} + \begin{pmatrix} g \\ 0 \end{pmatrix} &= \begin{pmatrix} \lambda \textbf S \Delta q \\ \lambda \end{pmatrix} \\
\textrm{or} \\
\begin{pmatrix} \textbf H & g \\ g^T & 0 \end{pmatrix} \begin{pmatrix} \Delta q \\ 1 \end{pmatrix} &= \lambda \begin{pmatrix} {\textbf{S} \Delta q} \\ 1 \end{pmatrix} \\
\end{align}

The matrix __S__ is usually taken to be the identify matrix.  The result is an eigenvalue equation with $\lambda$ as the eigenvalue!

$$
\begin{pmatrix} \textbf H & g \\ g^T & 0 \end{pmatrix}
\begin{pmatrix} \Delta q \\ 1 \end{pmatrix} = \lambda \begin{pmatrix} { \Delta q} \\ 1 \end{pmatrix}
$$


The $N+1$ dimensional matrix on the left is called the "RFO matrix".  We build it from our Hessian and our gradient in internal coordinates.  Then the eigenvectors and eigenvalues of this matrix are determined.  For minimum-energy searches, we usually choose the eigenvector with the lowest value of $\lambda$.  This value is hopefully negative, and is 2 times the  energy change anticipated from the step.  This eigenvector is intermediate-normalized by scaling the last element to 1.  The rest of the vector is the desired RFO step, or displacement in internal coordinates.

Occasionally, this eigenvector has a very small final element, and cannot be intermediate normalized.  This comes about, for example, when the step breaks molecular symmetry and the projected energy change may be zero.  Also, the algorithm is numerically problematic if the gradients are very small.

In practice, RFO performs better than ordinary Newton-Raphson steps if the energy surface being explored is not very harmonic, or if the optimization is begun far from a minimum.  However, if the potential surface is nearly flat (e.g., methane dimer), then RFO like ordinary N-R, will perform poorly.

## Demonstration
Now we prepare a water molecule and show an RFO step computation.

In [ ]:
mol = psi4.geometry("""
O
H 1 0.9
H 1 0.9 2 104
""")
# We'll use cc-pVDZ RHF.
psi4.set_options({"basis": "cc-pvdz"})
mol.update_geometry()

# Generate the internal coordinates manually.
intcos = [stre.STRE(0,1), stre.STRE(0,2), bend.BEND(1,0,2)]
for intco in intcos: 
    print(intco) 

# Handy variables for later.
Natom = mol.natom()
Nintco = len(intcos)
Z = [int(mol.Z(i)) for i in range(Natom)]

Now we guess the Hessian and compute the gradient in internal coordinates

In [ ]:
# Compute initial guess Hessian.
xyz = np.array(mol.geometry())

H = np.zeros((Nintco,Nintco), float)
for i,intco in enumerate(intcos):
    H[i,i] = intco.diagonalHessianGuess(xyz, Z, guessType="SCHLEGEL")

print("\n Schlegel Guess Hessian for Water (in au)")
print(H)

g_x = np.reshape( np.array( psi4.gradient('scf')), (3*Natom))
g_q = -1 * intcosMisc.qForces(intcos, xyz, g_x)

print("Gradient in internal coordinates (au)")
print(g_q)

Now we build the RFO matrix and diagonalize it.

In [ ]:
dim = Nintco + 1
RFOmat = np.zeros( (dim, dim), float)
RFOmat[0:-1,0:-1] = H
RFOmat[-1,0:-1] = g_q
RFOmat[0:-1,-1] = g_q
print("RFO matrix")
print(RFOmat)

evals, evects = linearAlgebra.symmMatEig(RFOmat)
print("Eigenvalues")
print(evals)

Happily, there is in this case only 1 direction with a negative predicted energy change.  We choose the corresponding eigenvector, and then intermediate normalize it.  That is our desired RFO step in internal coordinates!

In [ ]:
RFOdq = evects[0]
print("RFO eigenvector")
print(RFOdq)
RFOdq[:] = RFOdq / RFOdq[-1]
print("RFO intermediate normalized eigenvector")
print(RFOdq)
# Now drop the 1 at the end.
print("RFO step in internal coordinates")
RFOdq = RFOdq[0:-1]
print(RFOdq)

This is the desired step in internal coordinates.  We can see that the bond lengths and the bond angle will all increase.

## Extensions
The Partioned-RFO method is an extension of the RFO method, and may be used to seek non-minimum stationary points such as transition states.  In the P-RFO method, there are two RFO matrices.  One is used for maximization along 1 or more degrees of freedom while the second one is for minimization along the others.  For the P-RFO method to be effective the starting geometry must be reasonable, and the curvature of the surface in different directions must be correctly represented by the Hessian.  Thus, it is usually necessary to compute the Hessian for a transition-state optimization so that the algorithm begin maximizing in the right direction.

An extended scheme to limit the RFO step size called the restricted-step for RS-RFO method may be found in E. Besalu and J.M. Bofill, _Theor. Chem. Acc._ __100__, 265 (1998).  This involves an interation over repeated RFO matrix diagonalizations and does not always converge.  However, if it fails one simply resorts to more simple step-size scaling.